In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir())

['Untitled.ipynb', 'train.csv', 'sample_submission.csv', '.ipynb_checkpoints', 'test.csv', 'submission.csv']


In [2]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

In [3]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X_test = test_df.values

In [6]:
import tensorflow as tf

In [7]:
X = train_df.drop('label',axis=1)
Y = train_df['label']
Y = pd.get_dummies(Y)
X = X.values
Y = Y.values

In [8]:
x = tf.placeholder(tf.float32,[None, 784])
y = tf.placeholder(tf.float32,[None,10])
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500
n_classes = 10
batch_size = 100
X.shape[0]

42000

In [9]:
def neural_network_model(data):
    hidden_layer_1 = {'weights':tf.Variable(tf.random_normal([784,n_nodes_hl1])),
                     'bias':tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_layer_2 = {'weights':tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])),
                     'bias':tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_layer_3 = {'weights':tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])),
                     'bias':tf.Variable(tf.random_normal([n_nodes_hl3]))}
    output_layer = {'weights':tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])),
                     'bias':tf.Variable(tf.random_normal([n_classes]))}
    
    l1 = tf.matmul(data , hidden_layer_1['weights']) + hidden_layer_1['bias']
    l1 = tf.nn.relu(l1)
    l2 = tf.matmul(l1 , hidden_layer_2['weights']) + hidden_layer_2['bias']
    l2 = tf.nn.relu(l2)
    l3 = tf.matmul(l2, hidden_layer_3['weights']) + hidden_layer_3['bias']
    l3 = tf.nn.relu(l3)
    
    output = tf.matmul(l3,output_layer['weights']) + output_layer['bias']
    return output
    

In [10]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = prediction ))
    
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 100
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        total_batches = int(X.shape[0]/batch_size)
        for epoch in range(hm_epochs):
            epoch_loss=0
            
            for i in range(total_batches):
                epoch_x = X[i*100:(i+1)*100][:]
                epoch_y = Y[i*100:(i+1)*100][:]
                _, c = sess.run([optimizer,cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
            print("Epoch:", epoch , 'completed out of ', hm_epochs,' loss :' ,epoch_loss)
                  

        Pred = sess.run(prediction, feed_dict = {x:X_test})
    return Pred
            
                
                
                

In [11]:
Pred = train_neural_network(x)

Epoch: 0 completed out of  100  loss : 384872359.71875
Epoch: 1 completed out of  100  loss : 89911370.25390625
Epoch: 2 completed out of  100  loss : 49042741.125
Epoch: 3 completed out of  100  loss : 29019703.59753418
Epoch: 4 completed out of  100  loss : 16953888.70793152
Epoch: 5 completed out of  100  loss : 10619211.648132324
Epoch: 6 completed out of  100  loss : 6831083.167179108
Epoch: 7 completed out of  100  loss : 5587519.569205523
Epoch: 8 completed out of  100  loss : 3518878.9169692993
Epoch: 9 completed out of  100  loss : 3560365.7726745605
Epoch: 10 completed out of  100  loss : 2897600.016468048
Epoch: 11 completed out of  100  loss : 2580865.6782569885
Epoch: 12 completed out of  100  loss : 2909028.1832122803
Epoch: 13 completed out of  100  loss : 2896813.900123596
Epoch: 14 completed out of  100  loss : 2139374.8472709656
Epoch: 15 completed out of  100  loss : 2602081.240190267
Epoch: 16 completed out of  100  loss : 2989043.453135371
Epoch: 17 completed out o

In [12]:
type(Pred)

numpy.ndarray

In [13]:
Pred[0:1][:]

array([[-12651551. , -15932577. ,  26177114. ,  -2478352.2,  -1343832.6,
        -13201524. ,   2242715.8,  -8514935. ,  -9086143. , -13325747. ]],
      dtype=float32)

In [14]:
A = tf.argmax(Pred,1)
sess = tf.Session()
a = sess.run(A)

In [15]:
a = a.reshape(1,28000)
a.shape

(1, 28000)

In [16]:
df_output = pd.DataFrame(a)
df_output = df_output.T
df_output = df_output.reset_index()
df_output.columns = ['ImageId','Label']
df_output['ImageId'] = df_output['ImageId']+1
df_output.to_csv('submission.csv', index=False)